# Kaggle: SF Crime 

In [ ]:
# %matplotlib qt

In [21]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import preprocessing
# from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from os.path import expanduser, normpath
import time
import datetime

In [2]:
# Set paths for data to be imported

home = expanduser('~')
path = str(home) + '\\Documents\\data-science\\kaggle\\sf-crime\\' # Windows
# path = str(home) + '/Documents/Personal/data-science/kaggle/sf-crime/' # Mac
trainfile = 'train.csv'
testfile = 'test.csv'

In [3]:
train_data = pd.read_csv(path+trainfile)
test_data = pd.read_csv(path+testfile)

In [ ]:
# train_data.groupby('Category').size().sort_values()

## Features

In [4]:
train_data['Dates'] = pd.to_datetime(train_data['Dates'])
train_data['year'], train_data['month'], train_data['day'], train_data['hour'], train_data['minute'] = train_data['Dates'].dt.year, train_data['Dates'].dt.month, train_data['Dates'].dt.day, train_data['Dates'].dt.hour, train_data['Dates'].dt.minute

In [5]:
test_data['Dates'] = pd.to_datetime(test_data['Dates'])
test_data['year'], test_data['month'], test_data['day'], test_data['hour'], test_data['minute'] = test_data['Dates'].dt.year, test_data['Dates'].dt.month, test_data['Dates'].dt.day, test_data['Dates'].dt.hour, test_data['Dates'].dt.minute

In [6]:
# generate training and cross-validation features
X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(train_data.ix[:,['year','month','day','hour','minute','X','Y']], train_data.ix[:,'Category'], test_size=0.80, random_state=0)

In [7]:
X_test = test_data.ix[:,['year','month','day','hour','minute','X','Y']]

## Regression Model

In [8]:
# def MAPE(true, predicted):
#     return np.mean(np.abs((true - predicted)/true))

# def RMSE(error):
#     return (np.mean(error**2))**.05

In [9]:
# SUPPORT VECTOR MACHINE
t0 = time.time()

crime_svm = LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
crime_svm.fit(X_train, y_train)

y_train_pred = crime_svm.predict(X_train)
y_cv_pred = crime_svm.predict(X_cv)

t1 = time.time()
total = t1-t0
print (total) # 19 minutes

In [ ]:
# SUPPORT VECTOR MACHINE
t0 = time.time()

crime_svm = OneVsRestClassifier(SVC(kernel='rbf'))
crime_svm.fit(X_train, y_train)

y_train_pred = crime_svm.predict(X_train)
y_cv_pred = crime_svm.predict(X_cv)

t1 = time.time()
total = t1-t0
print (total)

In [15]:
t0 = time.time()

score_train = crime_svm.score(X_train, y_train)

t1 = time.time()
total = t1-t0
print ('Score:', score_train, ', Time:', total) # Score:  0.0880649625019 , Time:  0.27322959899902344

Score:  0.0880649625019 , Time:  0.27322959899902344


In [16]:
t0 = time.time()

score_cv = crime_svm.score(X_cv, y_cv)

t1 = time.time()
total = t1-t0
print ('Score:', score_cv, ', Time:', total) # Score:  0.0874252605205 , Time:  1.2154865264892578

Score:  0.0874252605205 , Time:  1.2154865264892578


In [18]:
t0 = time.time()

output = crime_svm.predict(X_test)

t1 = time.time()
total = t1-t0
print ('Time:', total) # Time: 0.3079204559326172

Time: 0.3079204559326172


In [19]:
pd.get_dummies(output)

,ASSAULT
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [ ]:
cat_uniques = train_data.Category.unique()

In [ ]:
headers = 'Id,' + ','.join(sorted(cat_uniques)) + '\n'
f = open('y_test.csv', 'w')
f.write(headers)
for i in xrange(len(y_test)):
    arr = [0] * 39
    arr[int(y_test)]